In [1]:
import pymongo
client = pymongo.MongoClient("localhost", 27017)


'test'

In [8]:
db = client.pyprotolinc
db.list_collection_names()

['results', 'jobs']

In [37]:
results_col = db.results
jobs_col = db.jobs

In [105]:
# delete all: 
#jobs_col.delete_many(filter={})
#results_col.delete_many(filter={})
for r in jobs_col.find():
    print(r)

{'_id': '10565964-e357-4ab9-b57a-4285d4890aeb', 'params': {'config_file': '/mnt/c/Programming/python/PyProtolinc/examples/03_mortality/config_for_cli_runner.yml'}, 'events': [{'type': 'JOB_RECEIVED', 'timestamp': 1682942065.111397}, {'hostname': 'celery@DESKTOP-F633F5A', 'utcoffset': -2, 'pid': 684, 'clock': 6637, 'uuid': '10565964-e357-4ab9-b57a-4285d4890aeb', 'name': 'pyprotolinc.server.tasks.valuation_run', 'args': "('/mnt/c/Programming/python/PyProtolinc/examples/03_mortality/config_for_cli_runner.yml',)", 'kwargs': '{}', 'root_id': '10565964-e357-4ab9-b57a-4285d4890aeb', 'parent_id': None, 'retries': 0, 'eta': None, 'expires': None, 'timestamp': 1682942065.125053, 'type': 'task-received', 'local_received': 1682942065.1295567, 'state': 'RECEIVED'}, {'hostname': 'celery@DESKTOP-F633F5A', 'utcoffset': -2, 'pid': 684, 'clock': 6638, 'uuid': '10565964-e357-4ab9-b57a-4285d4890aeb', 'timestamp': 1682942065.1323586, 'type': 'task-started', 'local_received': 1682942065.134752, 'state': 'ST

In [106]:
r["events"]

[{'type': 'JOB_RECEIVED', 'timestamp': 1682942065.111397},
 {'hostname': 'celery@DESKTOP-F633F5A',
  'utcoffset': -2,
  'pid': 684,
  'clock': 6637,
  'uuid': '10565964-e357-4ab9-b57a-4285d4890aeb',
  'name': 'pyprotolinc.server.tasks.valuation_run',
  'args': "('/mnt/c/Programming/python/PyProtolinc/examples/03_mortality/config_for_cli_runner.yml',)",
  'kwargs': '{}',
  'root_id': '10565964-e357-4ab9-b57a-4285d4890aeb',
  'parent_id': None,
  'retries': 0,
  'eta': None,
  'expires': None,
  'timestamp': 1682942065.125053,
  'type': 'task-received',
  'local_received': 1682942065.1295567,
  'state': 'RECEIVED'},
 {'hostname': 'celery@DESKTOP-F633F5A',
  'utcoffset': -2,
  'pid': 684,
  'clock': 6638,
  'uuid': '10565964-e357-4ab9-b57a-4285d4890aeb',
  'timestamp': 1682942065.1323586,
  'type': 'task-started',
  'local_received': 1682942065.134752,
  'state': 'STARTED'},
 {'hostname': 'celery@DESKTOP-F633F5A',
  'utcoffset': -2,
  'pid': 684,
  'clock': 6641,
  'uuid': '10565964-e357-

In [77]:

r["events"][-1]['timestamp']

1682935865.4772186

### Job Status Query

In [92]:
import pytz
from datetime import datetime

q = jobs_col.aggregate([{"$project": {"latestEvent": { "$last": "$events" }}},
                        {"$project": {"_id": 0, "job_id": "$_id", "state": "$latestEvent.state", "ts": "$latestEvent.timestamp"}}
                       ])

task_status = []
for job_info in list(q):
    task_status.append(job_info)
    ts = datetime.fromtimestamp(job_info["ts"], pytz.utc)
    job_info["ts"] = ts.strftime('%Y-%m-%d %H:%M:%S %Z%z')
    # ts.astimezone(pytz.timezone('Europe/Berlin')).strftime('%Y-%m-%d %H:%M:%S %Z%z')

task_status

[{'job_id': '16e6225d-69c9-4271-a988-9ab3d7d935f7',
  'state': 'SUCCESS',
  'ts': '2023-05-01 10:11:05 UTC+0000'}]

In [86]:
ts = datetime.fromtimestamp(r["events"][-1]['timestamp'], pytz.utc)
ts.astimezone(pytz.timezone('Europe/Berlin')).strftime('%Y-%m-%d %H:%M:%S %Z%z')

'2023-05-01 12:11:05 CEST+0200'

In [80]:
help(datetime.fromtimestamp)

Help on built-in function fromtimestamp:

fromtimestamp(...) method of builtins.type instance
    timestamp[, tz] -> tz's local time from POSIX timestamp.



In [99]:
from datetime import timezone
ts = datetime.now()
ts.replace(tzinfo=timezone.utc).timestamp()

1682949114.508494

In [100]:
help(datetime.now)

Help on built-in function now:

now(tz=None) method of builtins.type instance
    Returns new datetime object representing current time local to tz.
    
      tz
        Timezone object.
    
    If no tz is specified, uses local timezone.



In [103]:
ts = datetime.now(tz=timezone.utc).timestamp()
ts

1682941998.20702